This notebook regroups the code sample of the video below, which is a part of the [Hugging Face course](https://huggingface.co/course).

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/XAR8jnZZuUs?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

Install the Transformers and Datasets libraries to run this notebook.

In [11]:
! pip install datasets transformers[sentencepiece]

In [12]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")

def extract_languages(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    return {"inputs": inputs, "targets": targets}

raw_datasets = raw_datasets.map(extract_languages, batched=True, remove_columns=["id", "translation"])
raw_datasets

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 210173
    })
})

In [13]:
print(raw_datasets["train"][10])
print(raw_datasets["train"][11])
print(raw_datasets["train"][12])

{'inputs': 'translate', 'targets': 'traduction'}
{'inputs': 'The Babel & konqueror; plugin', 'targets': 'Le module externe Babel pour & konqueror;'}
{'inputs': 'Using the Babelfish plugin', 'targets': 'Utilisation du module externe Babelfish'}


In [14]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

sample = raw_datasets["train"][12]
inputs = tokenizer(sample["inputs"])
targets = tokenizer(sample["targets"])


print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))
print(tokenizer.convert_ids_to_tokens(targets["input_ids"]))

['▁Using', '▁the', '▁B', 'abel', 'fish', '▁plugin', '</s>']
['▁U', 'til', 'isation', '▁du', '▁module', '▁ex', 'tern', 'e', '▁B', 'abel', 'fish', '</s>']


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [15]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

sample = raw_datasets["train"][12]
inputs = tokenizer(sample["inputs"])
with tokenizer.as_target_tokenizer():
    targets = tokenizer(sample["targets"])


print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))
print(tokenizer.convert_ids_to_tokens(targets["input_ids"]))

['▁Using', '▁the', '▁B', 'abel', 'fish', '▁plugin', '</s>']
['▁Utilisation', '▁du', '▁module', '▁externe', '▁B', 'abel', 'f', 'ish', '</s>']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [16]:
max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    model_inputs = tokenizer(examples["inputs"], max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["targets"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(
    preprocess_function, batched=True, remove_columns=["inputs", "targets"]
)

In [18]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq

# Load the model and tokenizer
model_name = "t5-small"  # Replace with your model of choice
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)